In [1]:
# Adapted from LLama Factory
# https://colab.research.google.com/drive/1eRTPn37ltBbYsISy9Aw2NuI2Aq5CQrD9?usp=sharing#scrollTo=CS0Qk5OR0i4Q

In [2]:
!which python
!which pip

~/.conda/envs/py310/bin/python
~/.conda/envs/py310/bin/pip


In [3]:
#!git clone --depth 1 https://github.com/PepeRulo/LLaMA-Factory.git

In [5]:
%cd /research/rgs01/home/clusterHome/jpastr08/biohackathon/KIDS24-team12/vm_files/Jose/LLaMA-Factory
%pwd

/research/rgs01/home/clusterHome/jpastr08/biohackathon/KIDS24-team12/vm_files/Jose/LLaMA-Factory


'/research/rgs01/home/clusterHome/jpastr08/biohackathon/KIDS24-team12/vm_files/Jose/LLaMA-Factory'

In [6]:
# Commented out to be able to `Run All Cells` smoothly.

#!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1
#!pip uninstall -y jax
#!pip install -e .[torch,bitsandbytes,liger-kernel]
#!pip install --upgrade huggingface_hub

In [7]:
import sys
import torch

print(sys.version)
print(torch.__version__)

3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
2.3.1+cu121


In [8]:
import psutil

# Get the total RAM in bytes
total_ram = psutil.virtual_memory().total

# Convert bytes to GB
total_ram_gb = total_ram / (1024 ** 3)

print(f"Total RAM: {total_ram_gb:.2f} GB")


Total RAM: 1507.35 GB


In [9]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("Please set up a GPU before using LLaMA Factory: https://medium.com/mlearning-ai/training-yolov4-on-google-colab-316f8fff99c6")

device = torch.device("cuda")

# Get the name of the GPU
gpu_name = torch.cuda.get_device_name(device)
print(f"GPU: {gpu_name}")

# You can also get more details
print(f"Number of GPUs: {torch.cuda.device_count()}")
print(f"Current CUDA device: {torch.cuda.current_device()}")
print(f"CUDA capability: {torch.cuda.get_device_capability(device)}")

print(torch.__version__)
print(torch.version.cuda)

!nvidia-smi

GPU: NVIDIA A100-SXM4-80GB
Number of GPUs: 4
Current CUDA device: 0
CUDA capability: (8, 0)
2.3.1+cu121
12.1
Sun Dec  8 19:04:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:07:00.0 Off |                    0 |
| N/A   40C    P0              63W / 400W |      5MiB / 81920MiB |      0%      Default |
|                                         |      

In [12]:
import os
from huggingface_hub import login
with open("../hf_token.txt", 'r') as file:
    token = file.readline().strip()
    login(token=token)

In [22]:
# Don't forget to run this when there's been updates to the prompt dataset!
%cp /research/rgs01/home/clusterHome/jpastr08/biohackathon/KIDS24-team12/vm_files/Jose/files/dataset.json \
    ./data/dragen_alpaca.json

In [19]:
import json
with open("data/dataset_info.json", "r") as f:
    datasets = json.load(f)
    
# Per https://github.com/hiyouga/LLaMA-Factory/blob/main/data/README.md#alpaca-format
if "dragen_alpaca" not in datasets:
    datasets["dragen_alpaca"] = {
        "file_name": "dragen_alpaca.json",
        "columns": {
            "prompt": "instruction",
            "query": "input",
            "response": "output",
            "system": "system",
            "history": "history"
        }
    }
    
    with open("data/dataset_info.json", "w") as f:
        json.dump(datasets, f, indent=2, ensure_ascii=False)

In [20]:
## Run in terminal on the same node this OnDemand session is on.
# llamafactory-cli webui # by default it should launch a Gradio board on localhost:7860
## Run locally
# ssh -L <port shown above>:localhost:<port shown above> local_port_forwarding_test
## Open a browser locally to interface with Gradio board

In [60]:
# Training on Gradio board with
'''
llamafactory-cli train \
    --stage sft \
    --do_train True \
    --model_name_or_path meta-llama/Llama-3.2-90B-Vision-Instruct \
    --preprocessing_num_workers 16 \
    --finetuning_type lora \
    --template mllama \
    --flash_attn auto \
    --dataset_dir data \
    --dataset dragen_alpaca \
    --cutoff_len 2048 \
    --learning_rate 5e-05 \
    --num_train_epochs 7.0 \
    --max_samples 10000 \
    --per_device_train_batch_size 32 \
    --gradient_accumulation_steps 2 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 20 \
    --save_steps 750 \
    --warmup_steps 75 \
    --packing False \
    --report_to none \
    --output_dir saves/Llama-3.2-90B-Vision-Instruct/lora/train_2024-12-06-13-08-42 \
    --bf16 True \
    --plot_loss True \
    --ddp_timeout 180000000 \
    --optim adamw_torch \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.1 \
    --loraplus_lr_ratio 0.1 \
    --lora_target all \
    --val_size 0.1 \
    --eval_strategy steps \
    --eval_steps 750 \
    --per_device_eval_batch_size 32
'''

'\nllamafactory-cli train     --stage sft     --do_train True     --model_name_or_path meta-llama/Llama-3.2-90B-Vision-Instruct     --preprocessing_num_workers 16     --finetuning_type lora     --template mllama     --flash_attn auto     --dataset_dir data     --dataset dragen_alpaca     --cutoff_len 2048     --learning_rate 5e-05     --num_train_epochs 7.0     --max_samples 10000     --per_device_train_batch_size 32     --gradient_accumulation_steps 2     --lr_scheduler_type cosine     --max_grad_norm 1.0     --logging_steps 20     --save_steps 750     --warmup_steps 75     --packing False     --report_to none     --output_dir saves/Llama-3.2-90B-Vision-Instruct/lora/train_2024-12-06-13-08-42     --bf16 True     --plot_loss True     --ddp_timeout 180000000     --optim adamw_torch     --lora_rank 8     --lora_alpha 16     --lora_dropout 0.1     --loraplus_lr_ratio 0.1     --lora_target all     --val_size 0.1     --eval_strategy steps     --eval_steps 750     --per_device_eval_batch_siz

In [65]:
# Resources:
# https://www.reddit.com/r/LocalLLaMA/comments/1fps1cp/llama32_vs_llama31_in_medical_domain_llama31_70b/
# https://github.com/NVIDIA/RULER?tab=readme-ov-file